![tracker](https://us-central1-vertex-ai-mlops-369716.cloudfunctions.net/pixel-tracking?path=statmike%2Fvertex-ai-mlops%2FFramework+Workflows%2FKeras&file=Keras+with+JAX+-+Autoencoder+-+Dynamic+Specification.ipynb)
<!--- header table --->
<table align="left">
  <td style="text-align: center">
    <a href="https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/Keras/Keras%20with%20JAX%20-%20Autoencoder%20-%20Dynamic%20Specification.ipynb">
      <img width="32px" src="https://www.svgrepo.com/download/217753/github.svg" alt="GitHub logo">
      <br>View on<br>GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/Keras/Keras%20with%20JAX%20-%20Autoencoder%20-%20Dynamic%20Specification.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory logo">
      <br>Run in<br>Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https%3A%2F%2Fraw.githubusercontent.com%2Fstatmike%2Fvertex-ai-mlops%2Fmain%2FFramework%2520Workflows%2FKeras%2FKeras%2520with%2520JAX%2520-%2520Autoencoder%2520-%2520Dynamic%2520Specification.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise logo">
      <br>Run in<br>Colab Enterprise
    </a>
  </td>      
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/bigquery/import?url=https://github.com/statmike/vertex-ai-mlops/blob/main/Framework%20Workflows/Keras/Keras%20with%20JAX%20-%20Autoencoder%20-%20Dynamic%20Specification.ipynb">
      <img width="32px" src="https://www.gstatic.com/images/branding/gcpiconscolors/bigquery/v1/32px.svg" alt="BigQuery logo">
      <br>Open in<br>BigQuery Studio
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/statmike/vertex-ai-mlops/main/Framework%20Workflows/Keras/Keras%20with%20JAX%20-%20Autoencoder%20-%20Dynamic%20Specification.ipynb">
      <img width="32px" src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI logo">
      <br>Open in<br>Vertex AI Workbench
    </a>
  </td>
</table>

# Keras With JAX - Autoencoder - Dynamic Specification


Notes:

- supervised (maybe another workflow)
- semi-superfixed with pre-train and fine-tune (maybe another workflow)
- vae (another workflow)
- serving with continual/incrmental learning using mini batch update
- pipeline to train - include experiments, artifacts
    - set hyperparamters
    - train in component
    - train as training job
    - hyperparameter tuning to find best paramters with vizier.  Prepfer pipeline and direct api usage if possible.
- build autoencoder variations
    - fine tune as semi-supervised
    - fine tune as vae
    - fine tune as semi-supervised vae
- introduce timeseries autoencoder
    - fine tune as transformer using rows/time/series as sequence

---
## Dynamically Constructed Autoencoder

You can also build the autoencoder architecture dynamically using the size of the input data to specify the architecture.  The parameters used in dynamically specifying the architecture also make great targets for hyperparameter tuning exercises!

In [101]:
def build_dynamic_autoencoder(input_size, min_latent_factor=0.45, dropout_rate=0.4, l2_reg=0.001):
    """
    Builds a dynamic autoencoder with layer sizes that are powers of 2.

    Args:
        input_size: The number of input features.
        min_latent_factor: The minimum ratio of latent size to input size.
        dropout_rate: The dropout rate for each layer.
        l2_reg: The L2 regularization strength.

    Returns:
        A tuple: (autoencoder model, encoder model)
    """

    # Calculate the initial layer size (next power of 2 greater than input_size)
    initial_layer_size = 2**(math.ceil(math.log2(input_size)) + 1)
    min_latent_size = max(1, int(input_size * min_latent_factor))  # Ensure at least size 1

    # Determine the layers for the encoder
    encoder_layers = []
    current_size = initial_layer_size
    while current_size >= min_latent_size:
        encoder_layers.append(current_size)
        current_size //= 2  # Integer division to get the next power of 2

    # input
    autoencoder_input = keras.Input(shape=(input_size,), name="autoencoder_input")

    # Encoder
    x = autoencoder_input
    for i, size in enumerate(encoder_layers):
        x = keras.layers.Dense(size, activation='relu', name=f'enc_dense{i+1}', kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
        if dropout_rate > 0:  # Optionally add dropout
            x = keras.layers.Dropout(dropout_rate, name=f'enc_dropout{i+1}')(x)

    # Latent Space
    latent_size = encoder_layers[-1]
    latent = keras.layers.Dense(latent_size, activation='relu', name='latent', kernel_regularizer=keras.regularizers.l2(l2_reg))(x)

    # Decoder
    x = latent
    for i, size in reversed(list(enumerate(encoder_layers[:-1]))):
         x = keras.layers.Dense(size, activation='relu', name=f'dec_dense{i+1}', kernel_regularizer=keras.regularizers.l2(l2_reg))(x)
         if dropout_rate > 0:
            x = keras.layers.Dropout(dropout_rate, name=f'dec_dropout{i+1}')(x)

    # Add the output layer back in.
    reconstructed = keras.layers.Dense(input_size, activation='linear', name='reconstructed')(x)

    # Create Models
    autoencoder = keras.Model(inputs=autoencoder_input, outputs=reconstructed)
    encoder_model = keras.Model(inputs=autoencoder_input, outputs=latent)

    return autoencoder, encoder_model

In [102]:
autoencoder2, latent2 = build_dynamic_autoencoder(input_size = len(var_numeric))

In [103]:
autoencoder2.summary()

Model: "functional_6"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ autoencoder_input (InputLayer)  │ (None, 30)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense1 (Dense)              │ (None, 64)             │         1,984 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dropout1 (Dropout)          │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense2 (Dense)              │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dropout2 (Dropout)          │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dense3 (Dense)              │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ enc_dropout3 (Dropout)          │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ latent (Dense)                  │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_dense2 (Dense)              │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_dropout2 (Dropout)          │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_dense1 (Dense)              │ (None, 64)             │         2,112 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dec_dropout1 (Dropout)          │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ reconstructed (Dense)           │ (None, 30)             │         1,950 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 9,470 (36.99 KB)

 Trainable params: 9,470 (36.99 KB)

 Non-trainable params: 0 (0.00 B)

In [104]:
autoencoder2.compile(
    optimizer = keras.optimizers.Adam(learning_rate = 0.0005),
    loss = custom_loss, #keras.losses.MeanAbsoluteError(),
    metrics = [
        keras.metrics.RootMeanSquaredError(name = 'rmse'),
        keras.metrics.MeanSquaredError(name = 'mse'),
        keras.metrics.MeanAbsoluteError(name = 'mae'),
        keras.metrics.MeanSquaredLogarithmicError(name = 'msle')
    ]
)

In [ ]:
history2 = autoencoder2.fit(
    train_dataset,
    epochs = 50,
    validation_data = val_dataset,
    callbacks = [early_stopping]
)

Epoch 1/50
2275/2275 ━━━━━━━━━━━━━━━━━━━━ 39s 15ms/step - loss: 0.7265 - mae: 0.6566 - mse: 0.9896 - msle: 0.1404 - rmse: 0.9947 - val_loss: 0.6299 - val_mae: 0.6236 - val_mse: 0.9333 - val_msle: 0.1281 - val_rmse: 0.9661
Epoch 2/50
2275/2275 ━━━━━━━━━━━━━━━━━━━━ 41s 15ms/step - loss: 0.6371 - mae: 0.6305 - mse: 0.9624 - msle: 0.1291 - rmse: 0.9810 - val_loss: 0.6265 - val_mae: 0.6193 - val_mse: 0.9276 - val_msle: 0.1268 - val_rmse: 0.9631
Epoch 3/50
2275/2275 ━━━━━━━━━━━━━━━━━━━━ 44s 17ms/step - loss: 0.6341 - mae: 0.6266 - mse: 0.9487 - msle: 0.1274 - rmse: 0.9740 - val_loss: 0.6215 - val_mae: 0.6129 - val_mse: 0.9221 - val_msle: 0.1226 - val_rmse: 0.9603
Epoch 4/50
2275/2275 ━━━━━━━━━━━━━━━━━━━━ 39s 15ms/step - loss: 0.6307 - mae: 0.6219 - mse: 0.9440 - msle: 0.1243 - rmse: 0.9716 - val_loss: 0.6177 - val_mae: 0.6082 - val_mse: 0.9187 - val_msle: 0.1208 - val_rmse: 0.9585
Epoch 5/50
2275/2275 ━━━━━━━━━━━━━━━━━━━━ 38s 15ms/step - loss: 0.6278 - mae: 0.6181 - mse: 0.9390 - msle: 0.122